In [1]:
import tensorflow as tf
from keras import backend as K
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
tf.config.list_physical_devices('GPU')

2023-02-03 21:06:38.453677: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Step 1 加载数据

In [2]:
data = pd.read_csv('../data/Sohu2022_data/rec_data/train-dataset.csv')
# test_data = pd.read_csv('../data/Sohu2022_data/rec_data/test-dataset.csv') 

Step 2 特征工程

将类别列重新映射

In [3]:
#需要做映射的列
dict_cols = [x for x in data.columns if x not in ['sampleId','label','pvId','userSeq','logTs',]]

In [4]:
names = globals()

In [5]:
from tqdm import tqdm
for col in tqdm(dict_cols):
    #基础数据
    names[str(col)+'_values' ] = list(data[col].unique())
    #基础映射表,维护
    names[str(col)+'_dict' ] = {value:idx for idx,value in enumerate(names[str(col)+'_values' ])}
    #字典大小
    names[str(col)+'_dict_size'] = len(names[str(col)+'_dict' ])
    #映射到data
    data[f"{col}_idx"] = data[col].map(names[str(col)+'_dict' ])

#如果需要添加新数据扩大映射表，参考特征工程.ipynb

100%|██████████| 8/8 [00:01<00:00,  6.53it/s]


对logTs进行处理

In [6]:
data['date'] = pd.to_datetime(data['logTs'], unit='ms') 
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

超参数

In [7]:
EMBEEDING_SIZE = 64

Step 3 制作inputs

In [8]:
inputs = {
    'suv' : tf.keras.layers.Input(name='suv', shape=(), dtype='int32'),
    'operator' : tf.keras.layers.Input(name='operator', shape=(), dtype='int32'),
    'browserType' : tf.keras.layers.Input(name='browserType', shape=(), dtype='int32'),
    'deviceType' : tf.keras.layers.Input(name='deviceType', shape=(), dtype='int32'),
    'osType' : tf.keras.layers.Input(name='osType', shape=(), dtype='int32'),
    'province' : tf.keras.layers.Input(name='province', shape=(), dtype='int32'),
    'itemId' : tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
    'city' : tf.keras.layers.Input(name='city', shape=(), dtype='int32'),
}

Step 4 搭建模型

In [9]:
itemId_embedding = tf.keras.layers.Embedding(itemId_dict_size+1, EMBEEDING_SIZE )(inputs['itemId'])
suv_embedding = tf.keras.layers.Embedding(suv_dict_size+1, EMBEEDING_SIZE )(inputs['suv'])
operator_embedding = tf.keras.layers.Embedding(operator_dict_size+1, EMBEEDING_SIZE )(inputs['operator'])
browserType_embedding = tf.keras.layers.Embedding(browserType_dict_size+1, EMBEEDING_SIZE )(inputs['browserType'])
deviceType_embedding = tf.keras.layers.Embedding(deviceType_dict_size+1, EMBEEDING_SIZE )(inputs['deviceType'])
osType_embedding = tf.keras.layers.Embedding(osType_dict_size+1, EMBEEDING_SIZE )(inputs['osType'])
province_embedding = tf.keras.layers.Embedding(province_dict_size+1, EMBEEDING_SIZE )(inputs['province'])
city_embedding = tf.keras.layers.Embedding(city_dict_size+1, EMBEEDING_SIZE )(inputs['city'])

2023-02-03 21:06:55.977779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 21:06:56.643890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22350 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:56:00.0, compute capability: 8.6


In [10]:
all_features = tf.keras.layers.Concatenate(axis=-1)([itemId_embedding,suv_embedding,
                                            operator_embedding,browserType_embedding,
                                            deviceType_embedding,osType_embedding,
                                            province_embedding,city_embedding])

In [11]:
class cross_layer(tf.keras.layers.Layer):
    def __init__(self,input_dim,output_dim) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
    def build(self,input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.input_dim, self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(cross_layer, self).build(input_shape)
    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "output_dim": self.output_dim,
        })
        return config
        
    def call(self,x):
        formula_left = K.pow(K.dot(x,self.kernel),2)
        formula_right = K.dot(K.pow(x,2),K.pow(self.kernel,2))
        return  0.5*(formula_left-formula_right)

In [12]:
#fm
linear = tf.keras.layers.Dense(1,activation='relu')(all_features)
cross = cross_layer(input_dim=512,output_dim=64)(all_features)
fm_output = tf.keras.layers.Concatenate(axis=-1)([linear,cross])

In [13]:
#deep
deep = tf.keras.layers.Dense(256,activation='relu')(all_features)
deep_output = tf.keras.layers.Dense(64,activation='relu')(deep)

In [14]:
outputs = tf.keras.layers.Concatenate(axis=-1)([fm_output,deep_output])
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(outputs)

In [15]:
model = tf.keras.Model(inputs,outputs)

In [16]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=tf.keras.metrics.AUC(name='auc'))

Step 5 划分数据集：训练集：验证集：测试集 = 6：2：2

In [17]:
#训练集：验证集:测试集 = 6：2：2
import random
train_len = int(len(data)*0.6)
val_len = int(len(data)*0.2)
idx = list(data.index)
random.shuffle(idx)
#训练集
df_train = data.iloc[idx[:train_len]]
#验证集
df_val = data.iloc[idx[train_len:train_len+val_len]]
#测试集
df_test = data.iloc[idx[train_len+val_len:]]

In [18]:
x_train = [df_train['suv_idx'],
           df_train['operator_idx'],
           df_train['browserType_idx'],
           df_train['deviceType_idx'],
           df_train['osType_idx'],
           df_train['province_idx'],
           df_train['itemId_idx'],
           df_train['city_idx']]
y_train = df_train['label']

x_val = [df_val['suv_idx'],
         df_val['operator_idx'],
         df_val['browserType_idx'],
         df_val['deviceType_idx'],
         df_val['osType_idx'],
         df_val['province_idx'],
         df_val['itemId_idx'],
         df_val['city_idx']]
y_val = df_val['label']

x_test = [df_test['suv_idx'],
          df_test['operator_idx'],
          df_test['browserType_idx'],
          df_test['deviceType_idx'],
          df_test['osType_idx'],
          df_test['province_idx'],
          df_test['itemId_idx'],
          df_test['city_idx']]
y_test = df_test['label']

In [19]:
from datetime import datetime
TIMESTAMP = "{0:%Y-%m-%dT%H-%M-%S/}".format(datetime.now())
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="../logs/Deep_FM"+TIMESTAMP)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('../save/DeepFM/{epoch:02d}-{val_loss:.2f}.hdf5',monitor='val_loss')              
history = model.fit(x_train,y_train,epochs=1,batch_size=32,
                    validation_data=[x_val,y_val],
                    callbacks=[tensorboard_callback,checkpoint_callback,
                    ])

   15/79384 [..............................] - ETA: 10:43 - loss: 0.6778 - auc: 0.4890 

2023-02-03 21:07:08.300193: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


79384/79384 [==============================] - 1208s 15ms/step - loss: 0.4895 - auc: 0.5259 - val_loss: 0.4898 - val_auc: 0.5322


In [20]:
model.evaluate(x_test, y_test, batch_size=32)

26462/26462 [==============================] - 273s 10ms/step - loss: 0.4897 - auc: 0.5297


[0.4896877706050873, 0.5296633839607239]